In [9]:
# 

Annotation file reader

In [10]:
from pycocotools.coco import COCO

ann_file = "C:\\Users\\S_G\Desktop\\yolo_dataset\\coco\\annotations_trainval2017\\annotations\\instances_val2017.json"
ann_file = "C:\\Users\\S_G\\Documents\\GitHub\\Shape_Detection\\3_shape_annotation.json"
coco=COCO(ann_file)
category_ids = coco.getCatIds(['class0'])


loading annotations into memory...
Done (t=0.00s)
creating index...


KeyError: 'image_id'

In [ ]:
image_ids = coco.getImgIds(catIds=[2])
print("id",image_ids[0:5])
annotation_ids = coco.getAnnIds(imgIds=169996, catIds=[2])
print(len(annotation_ids))

In [ ]:
anns = coco.loadAnns(annotation_ids)
for ann in anns:
    print(ann['bbox'])

[151.74, 343.74, 108.39, 96.0]
[399.89, 329.15, 81.47, 98.39]
